In [1]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
!pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.3

#Importing GIS Software
from rasterio import windows
from rasterio.features import shapes
from  gis_utils import raster
import geopandas as gpd
import rasterio 
import gdal 
import os

In [2]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [3]:
from tensorflow.keras.layers import InputLayer, Dense, concatenate
%load_ext tensorboard

In [4]:
#loading training data

from google.colab import drive
drive.mount('/content/drive')
 
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Training Images/Hybrid CNN/Planetscope+NDVI_allsites.zip" .
!unzip "/content/Planetscope+NDVI_allsites.zip" -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/1/lr_91479.tif  
  inflating: dataset/1/lr_9148.tif   
  inflating: dataset/1/lr_91480.tif  
  inflating: dataset/1/lr_91481.tif  
  inflating: dataset/1/lr_91482.tif  
  inflating: dataset/1/lr_91483.tif  
  inflating: dataset/1/lr_91484.tif  
  inflating: dataset/1/lr_91485.tif  
  inflating: dataset/1/lr_91486.tif  
  inflating: dataset/1/lr_91487.tif  
  inflating: dataset/1/lr_91488.tif  
  inflating: dataset/1/lr_91489.tif  
  inflating: dataset/1/lr_9149.tif   
  inflating: dataset/1/lr_91490.tif  
  inflating: dataset/1/lr_91491.tif  
  inflating: dataset/1/lr_91492.tif  
  inflating: dataset/1/lr_91493.tif  
  inflating: dataset/1/lr_91494.tif  
  inflating: dataset/1/lr_91495.tif  
  inflating: dataset/1/lr_91496.tif  
  inflating: dataset/1/lr_91497.tif  
  inflating: dataset/1/lr_91498.tif  
  inflating: dataset/1/lr_91499.tif  
  inflating: dataset/1/lr_9150.tif   
  inflating: dataset/1/lr_91500.tif  

In [5]:
from skimage.transform import resize
from os.path import basename, dirname
from rasterio.plot import reshape_as_image
import glob

def load_images(image_dir):

    num_images = (len(glob.glob1("/content/dataset/1","*.tif")) + len(glob.glob1("/content/dataset/0","*.tif")))/2
    hr_mem = np.memmap('hr.memmap', dtype='float32', mode='w+', shape=(int(num_images),85,85,3))
    lr_mem = np.memmap('lr.memmap', dtype='float32', mode='w+', shape=(int(num_images),5))

    index = 0
    labels = []

    for root, dirs, files in os.walk(image_dir):
        for file in tqdm(files):
            if (file.endswith(".tif") and file.startswith("hr_")):
                file_path = os.path.join(root, file)
                
                hr_image, _ = raster.load_image(file_path)
                lr_image, _ = raster.load_image(file_path.replace("hr_", "lr_"))
                label = int(basename(dirname(file_path)))
                
                labels.append(label)
                hr_mem[index,:,:,:] = resize(reshape_as_image(hr_image.read()), (85,85,3))
                lr_mem[index,:] = np.squeeze(np.squeeze(reshape_as_image(lr_image.read())))
                index += 1

    labels_array = np.asarray(labels)
    del labels

    return hr_mem, lr_mem, labels_array

In [6]:
hr, lr, labels = load_images("dataset")

print(hr.shape)
print(lr.shape)
print(labels.shape)


(340969, 85, 85, 3)
(340969, 5)
(340969,)


In [7]:
def create_cnn(module_handle, module_name, image_size, n_2):
    model = tf.keras.Sequential([
    InputLayer(input_shape = IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable = False, name = module_name),
    Dense(50)])
    return model

In [8]:
def create_mlp(n_channels, n_1):
    model = tf.keras.Sequential([
       tf.keras.layers.Dense(32, input_dim=n_channels)                         
    ])
    return model

In [9]:
def show_model(model):
    tf.keras.utils.plot_model(model)
    model.summary()

In [10]:
def create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2):
    feature_extractor = create_cnn(MODULE_HANDLE,"Resnet50V2", IMAGE_SIZE,n_2)
    mlp = create_mlp(5,n_1)

    combined = concatenate([mlp.output, feature_extractor.output])
    x = Dense(2, activation="sigmoid")(combined)


    model = tf.keras.Sequential()
    model = tf.keras.models.Model(inputs=[mlp.input, feature_extractor.input], outputs=x)
   
    model.compile(
    optimizer=tf.keras.optimizers.RMSprop(), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

    return model

In [11]:
def create_base(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE):
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    tf.keras.layers.Dense(2, activation="sigmoid")
    ])
    model.build((None,)+IMAGE_SIZE+(3,))
    return model


In [17]:
from sklearn.model_selection import train_test_split

#getting indices of train and test samples to avoid memory issues
num_images = (len(glob.glob1("/content/dataset/1","*.tif")) + len(glob.glob1("/content/dataset/0","*.tif")))/2
x_ids = list(range(int(num_images)))
x_train_ids, x_test_ids, Y_train, Y_test = train_test_split(x_ids, labels, test_size = 0.2, random_state=42)

In [ ]:
#splitting data into new memmaps 
x_hr_train = np.memmap('hr_train.memmap', dtype='float32', mode='w+', shape=(len(x_train_ids),85,85,3))
x_hr_test = np.memmap('hr_test.memmap', dtype='float32', mode='w+', shape=(len(x_test_ids),85,85,3))
x_lr_train = np.memmap('lr_train.memmap', dtype='float32', mode='w+', shape=(len(x_train_ids),lr.shape[1]))
x_lr_test = np.memmap('lr_test.memmap', dtype='float32', mode='w+', shape=(len(x_test_ids),lr.shape[1]))

for i, index in tqdm(enumerate(x_train_ids)):
     x_hr_train[i,:,:,:] = hr[index,:,:,:]
     x_lr_train[i,:] = lr[index,:]


for i, index in tqdm(enumerate(x_test_ids)):
     x_hr_train[i,:,:,:] = hr[index,:,:,:]
     x_lr_train[i,:] = lr[index,:]

In [ ]:
x_hr_train = np.memmap('hr_train.memmap', dtype='float32', mode='w+', shape=(int(num_images*.8),85,85,3))
x_hr_test = np.memmap('hr_test.memmap', dtype='float32', mode='w+', shape=(int(num_images*.2),85,85,3))
indices = np.arange(labels.shape[0])
x_hr_train[:,:,:,:], x_hr_test[:,:,:,:], y_train, y_test, idx_train, idx_test = train_test_split(hr, labels, indices, test_size=0.2, random_state=1 )
x_lr_train = np.take(lr, idx_train, axis = 0)
x_lr_test = np.take(lr, idx_test, axis=0)

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

In [ ]:
import pandas as pd
columns = ["max_accuracy","epoch","n_1","n_2"]

results = pd.DataFrame(columns = columns)
results

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMAGE_SIZE = (85,85)
BATCH_SIZE = 16

n_1=32
n_2=256
model = create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2)
history = model.fit(x = [x_lr_train, x_hr_train], y = y_train, validation_data= ([x_lr_test, x_hr_test], y_test), epochs = 30)
model.save("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/efficientnetb0-pretrained-full")

In [ ]:
model.save("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/resisc45-resnet")

INFO:tensorflow:Assets written to: /content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/resisc45-resnet/assets


INFO:tensorflow:Assets written to: /content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/resisc45-resnet/assets
